<a href="https://colab.research.google.com/github/SebastianLarssonDTU/02456-Reinforcement-Learning-Project/blob/restructure_code/StarpilotProject_Implementing_evalutation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INIT : Procgen, Drive, Git

In [ ]:
!pip install procgen

In [ ]:
#Clone git
!git clone -b restructure_code https://github.com/SebastianLarssonDTU/02456-Reinforcement-Learning-Project.git "my_project"

In [ ]:
#update git
%cd /content/my_project
! git pull

In [ ]:
import datatools as tools
from datatools import DATA_PATH, MODEL_PATH
#Mount drive
tools.mount_drive()

# TESTING IMPLEMENTATION

In [ ]:
from baseline import set_hyperparameters
import hyperparameters as h
from ppo import PPO
import torch

In [ ]:
set_hyperparameters(baseline='Procgen')
model = PPO(print_output=True)

In [ ]:
policy = model.train()

In [ ]:
tools.create_index_table_from_txt_files()

# Testing evaluation

In [ ]:
from model import Encoder
from policy import Policy
import utils
import hyperparameters as h
import torch.nn as nn
import imageio

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)



MODEL_PATH = '/content/drive/My Drive/02456-Deep-Learning-Project/Models/'

def load_policy(file_name):
  encoder = Encoder(in_channels = h.in_channels, feature_dim = h.feature_dim)  
  
  policy = Policy(encoder = encoder, feature_dim = h.feature_dim, num_actions = 15)
  policy.cuda()
  policy.load_state_dict(torch.load(MODEL_PATH + file_name + '.pt')["policy_state_dict"])
  policy.cuda() 

  return policy

In [ ]:
def policy_evaluation(video_name='vid', print_output=False, policy=None, file_name=None):
  if policy is None and file_name is None:
    raise ValueError("Policy should be specified by file_name or given as object")
  if policy is None:
    policy = load_policy(file_name)
  
  # Make evaluation environment
  eval_env = utils.make_env(h.num_envs, start_level=h.num_levels, num_levels=h.num_levels)
  obs = eval_env.reset()

  frames = []
  total_reward = []
  
  # Evaluate policy
  policy.eval()
  for _ in range(512):  #<--- Hardcoded batch size?

    # Use policy
    action, log_prob, value = policy.act(obs)

    # Take step in environment
    obs, reward, done, info = eval_env.step(action)
    total_reward.append(torch.Tensor(reward))

    # Render environment and store
    frame = (torch.Tensor(eval_env.render(mode='rgb_array'))*255.).byte()
    frames.append(frame)

  # Calculate average return
  total_reward = torch.stack(total_reward).sum(0).mean(0)
  
  if print_output:
    print('Average return:', total_reward)

  # Save frames as video
  frames = torch.stack(frames).cpu().numpy()
  imageio.mimsave(video_name+'.mp4', frames, fps=25)
  
  return total_reward

In [ ]:
INDEX_TABLE = tools.create_index_table_from_txt_files()
file_name = INDEX_TABLE['file_name'][-1].strip()

policy = load_policy(file_name)
policy_evaluation(print_output=True, policy = policy)